Before we start, we need to make sure that we have a Kafka cluster running and a topic that produces some streaming data. For simplicity, we will use a single-node Kafka cluster and a topic named `users`. Open the `5.0 user-gen-kafka.ipynb` notebook and execute the cell. This notebook produces a user record every few seconds and put it on a Kafka topic called `users`. 

In [1]:
from delta import  # Delta Lake 라이브러리 임포트 configure_spark_with_delta_pip, DeltaTable
from pyspark.sql import SparkSession  # Spark SQL 작업을 위한 SparkSession 임포트
from pyspark.sql.functions import  # Spark SQL 함수들 임포트 col, from_json
from pyspark.sql.types import  # Spark SQL 데이터 타입 임포트 StructType, StructField, IntegerType, StringType

In [2]:
builder = (SparkSession.builder  # SparkSession 빌더 패턴 시작
           .appName("delta-write-streaming")  # 애플리케이션 이름 설정
           .master("spark://spark-master:7077")  # Spark 마스터 URL 설정
           .config("spark.executor.memory", "512m")  # Spark 설정 옵션
           .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")  # Spark 설정 옵션
           .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")  # Spark 설정 옵션)

spark = configure_spark_with_delta_pip(builder,['org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1']).getOrCreate()  # SparkSession 생성 또는 기존 세션 반환
spark.sparkContext.setLogLevel("ERROR")  # 로그 레벨을 ERROR로 설정

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c4dcb31a-7c43-4517-a7a2-78df7f6dcbd0;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.4.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.4.1 in central
	found org.apache.kafka#kafka-clients;3.3.2 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.1 in central
	found org.slf4j#slf4j-api;2.0.6 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in centra

In [3]:
get_ipython().run_line_magic('load_ext', 'sparksql_magic')
get_ipython().run_line_magic('config', 'SparkSql.limit=20')

In [4]:
%%sparksql
CREATE OR REPLACE TABLE default.users (
    id INT,
    name STRING,
    age INT,
    gender STRING,
    country STRING 
) USING DELTA  # Delta Lake 테이블 생성 LOCATION '/opt/workspace/data/delta_lake/delta-write-streaming/users';

In [5]:
df = (spark.readStream  # 스트리밍 데이터 읽기
      .format("kafka")
      .option("kafka.bootstrap.servers", "kafka:9092")
      .option("subscribe", "users")
      .option("startingOffsets", "earliest")
      .load(  # 파일 로드))

In [6]:
schema = StructType  # 구조체 타입([
    StructField  # 구조체 필드('id', IntegerType(), True),
    StructField  # 구조체 필드('name', StringType(), True),
    StructField  # 구조체 필드('age', IntegerType(), True),
    StructField  # 구조체 필드('gender', StringType(), True),
    StructField  # 구조체 필드('country', StringType(), True)])

df = df.withColumn(  # 새 컬럼 추가 또는 기존 컬럼 수정'value', from_json(col('value').cast("STRING"), schema))

In [7]:
df = df.select(  # 컬럼 선택
    col(  # 컬럼 참조'value.id').alias('id'),
    col(  # 컬럼 참조'value.name').alias('name'),
    col(  # 컬럼 참조'value.age').alias('age'),
    col(  # 컬럼 참조'value.gender').alias('gender'),
    col(  # 컬럼 참조'value.country').alias('country'))

In [8]:
query = (df.writeStream  # 스트리밍 데이터 쓰기
   .format("delta")  # Delta Lake 형식으로 저장
   .outputMode(  # 스트리밍 출력 모드 설정"append")
   .option("checkpointLocation", "/opt/workspace/data/delta_lake/delta-write-streaming/users/_checkpoints/")
   .start("/opt/workspace/data/delta_lake/delta-write-streaming/users"))

In [9]:
%%sparksql
SELECT COUNT(*) FROM delta.`/opt/workspace/data/delta_lake/delta-write-streaming/users`;

count(1)
118


In [10]:
%%sparksql
SELECT COUNT(*) FROM delta.`/opt/workspace/data/delta_lake/delta-write-streaming/users`;

count(1)
120


In [11]:
query.stop()

In [12]:
spark.stop()  # Spark 세션 종료 - 리소스 정리 